In [1]:
import pandas as pd
data = pd.read_csv("medical_symptoms.csv")
print(data.columns.tolist())


['Domain', 'Type', 'Symptoms']


In [3]:
!pip install joblib



In [2]:
from flask import Flask, render_template, request, jsonify, session
import pandas as pd
import os
import pickle

app = Flask(__name__)
app.secret_key = "secret123"  # for session management

# Load the model and encoders
model, mlb, label_encoder = None, None, None
try:
    with open("auto_model.pkl", "rb") as f:
        model, mlb, label_encoder = pickle.load(f)
    print("✅ Auto model loaded")
except Exception as e:
    print("❌ Failed to load ANN model:", e)


# Load medical_symptoms.csv
df_symptoms = None
try:
    CSV_PATH = os.path.join(os.getcwd(), "medical_symptoms.csv")
    df_symptoms = pd.read_csv(CSV_PATH)
    print("✅ medical_symptoms.csv loaded")
except Exception as e:
    print("❌ Failed to load medical_symptoms.csv:", e)


@app.route("/")
def home():
    session.clear()
    # Pass symptom list to HTML (optional)
    symptoms = df_symptoms["Symptom"].tolist() if df_symptoms is not None else []
    return render_template("index.html", symptoms=symptoms)

@app.route("/chat", methods=["POST"])
def chat():
    user_input = request.form.get("message", "").strip().lower()

    if model is None:
        return jsonify({"reply": "⚠️ ANN model not available. Try again later."})

    # Step 1: Ask for duration
    if "step" not in session:
        session["symptoms"] = user_input
        session["step"] = "ask_duration"
        return jsonify({"reply": "🕒 How long have you been experiencing these symptoms?"})

    # Step 2: Ask for medical history
    elif session["step"] == "ask_duration":
        session["duration"] = user_input
        session["step"] = "ask_history"
        return jsonify({"reply": "🩺 Do you have any previous medical history? If yes, please mention."})

    # Step 3: Predict disease
    elif session["step"] == "ask_history":
        session["history"] = user_input
        symptoms_list = [s.strip() for s in session["symptoms"].split(",")]

        try:
            input_vector = mlb.transform([symptoms_list])
            prediction = model.predict(input_vector)
            disease = label_encoder.inverse_transform(prediction)[0]

            response = f"""
            🩺 Based on the symptoms you’ve described ({session['symptoms']}),<br>
            duration ({session['duration']}), and history ({session['history']}),<br>
            you might be showing signs of: <strong>{disease}</strong>.<br><br>
            I am not 100% sure since I’m an AI assistant 🤖. For more clarity, please visit our website or consult a certified doctor. 🏥
            """
        except Exception as e:
            response = f"⚠️ Error while predicting disease: {str(e)}"

        session.clear()
        return jsonify({"reply": response})

    else:
        session.clear()
        return jsonify({"reply": "🔄 Let's start again. Please tell me your symptoms (comma separated)."})

if __name__ == "__main__":
    app.run(debug=True)


✅ Auto model loaded
✅ medical_symptoms.csv loaded
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\SRIDEEPALAKSHMI\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
